In [1]:
import os
import librosa
import pandas as pd

In [3]:
import os
import numpy as np
import librosa

def preprocess_audio_files(data_folder):
    sr = 44100  # sample rate
    max_duration = 0  # max duration in seconds
    max_length = 0  # max length in samples
    sr = 44100

    # First, find the longest audio file
    subfolders = os.listdir(data_folder)

    for i, subfolder in enumerate(subfolders):
        subfolder_path = os.path.join(data_folder, subfolder)

        if not os.path.isdir(subfolder_path):
            continue

        for filename in os.listdir(subfolder_path):
            if filename.endswith('.wav'):
                file_path = os.path.join(subfolder_path, filename)

                # Load the audio file
                y, sr = librosa.load(file_path, sr=sr)

                # Get duration
                duration = librosa.get_duration(y=y, sr=sr)

                if duration > max_duration:
                    max_duration = duration
                    max_length = len(y)

    max_length = int(max_length)  # ensure max_length is an integer

    # Next, pad all audio files to match the length of the longest file
    for i, subfolder in enumerate(subfolders):
        subfolder_path = os.path.join(data_folder, subfolder)

        if not os.path.isdir(subfolder_path):
            continue

        for filename in os.listdir(subfolder_path):
            if filename.endswith('.wav'):
                file_path = os.path.join(subfolder_path, filename)

                # Load the audio file
                y, sr = librosa.load(file_path, sr=sr)

                # Pad the audio file to max_length
                if len(y) < max_length:
                    y_padded = np.pad(y, (0, max_length - len(y)))  # pads with zeros at the end
                else:
                    y_padded = y

                # Save the padded audio file
                padded_file_path = os.path.join(subfolder_path, f'padded_{filename}')
                librosa.output.write_wav(padded_file_path, y_padded, sr)

    print(f'Max duration: {max_duration} seconds')
    print(f'Max length: {max_length} samples')

# Running the function
data_folder = r"C:\Users\GHB\Desktop\SCREW PROJECT\Machine Learning\Final data\Extrinsic data (clean)"
preprocess_audio_files(data_folder)


AttributeError: module 'librosa' has no attribute 'load'

In [4]:
import os
import glob
import librosa
import numpy as np

# Find all audio files and labels
audio_files = []
labels = []

for root, dirs, files in os.walk(r"C:\Users\GHB\Desktop\SCREW PROJECT\Machine Learning\Final data\Extrinsic data (clean)"):  # Replace '.' with your top-level directory
    for file in files:
        if file.endswith('.wav'):  # Adjust this if you're using a different file format
            audio_files.append(os.path.join(root, file))
            labels.append(os.path.basename(root))  # Label is the base name of the directory containing the file

# Find the longest audio file
max_length = 0
for audio_file in audio_files:
    audio, sr = librosa.load(audio_file)
    if len(audio) > max_length:
        max_length = len(audio)
max_length_seconds = max_length / sr  # Convert from samples to seconds

# Define the padding function
def pad_audio(audio_path, target_length, sr=44100):
    audio, _ = librosa.load(audio_path, sr=sr)  # Load audio
    length = len(audio)
    target_samples = int(target_length * sr)  # Calculate the target number of samples

    if length < target_samples:  # If the audio is shorter than the target length
        # Pad the audio with zeros at the end
        audio = np.concatenate([audio, np.zeros(target_samples - length)])
    elif length > target_samples:  # If the audio is longer than the target length
        # Trim the audio to the target length
        audio = audio[:target_samples]

    return audio

# Pad all audio files to the length of the longest audio file
padded_audio_files = []
for audio_file in audio_files:
    padded_audio = pad_audio(audio_file, max_length_seconds, sr)
    padded_audio_files.append(padded_audio)


AttributeError: module 'librosa' has no attribute 'load'

In [9]:
print(padded_audio_files[0])
print(labels[0])

[-6.28574881e-08 -1.60502241e-05  2.53893813e-06 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
N


In [13]:
# Save data
np.savez(r'C:\Users\GHB\Desktop\SCREW PROJECT\Machine Learning\Deep learning\Extrinsic\padded_audio (clean).npz', x=padded_audio_files, y=labels)